In [1]:
import numpy as np
import tensorflow as tf
import matplotlib
#matplotlib.use('Agg')
import matplotlib.pyplot as plt
import pickle as pkl


import tensorflow_probability as tfp

np.random.seed(0)
tf.random.set_seed(0)

In [2]:
DIM = 64 # Model dimensionality (number of neurons in the hidden layer(s))
CRITIC_ITERS = 10 # How many critic iterations (Sinkhorn iterations) per generator iteration#was 50
BATCH_SIZE = 1024#256 # Batch size
ITERS = 2500#100000 # how many generator iterations to train for
DATA_DIM = 32
LATENT_DIM = 4
INITIALIZATION = 'he'#'glorot'
COVARIANCE_SCALE = np.sqrt(DATA_DIM)
INITIALIZE_LAST = True
SAMPLE_SIZE = 100000
LAMBDA = 0.3#2/(COVARIANCE_SCALE)
MODE = 'divergence' #'loss'

In [3]:
from tensorflow import keras
from tensorflow.keras import layers

In [4]:
if INITIALIZATION == 'he':
    weight_initializer = keras.initializers.he_uniform()
if INITIALIZATION == 'glorot':
    weight_initializer = keras.initializers.glorot_uniform()
bias_initializer = keras.initializers.zeros()

In [5]:
#class WGAN:
 #   def __init__()self
latent_sample = keras.Input(shape=(LATENT_DIM,))
hidden_layer = latent_sample
for i in range(3):
    hidden_layer = layers.Dense(DIM, activation="relu", kernel_initializer=weight_initializer,
        bias_initializer=bias_initializer)(hidden_layer)
output = layers.Dense(DATA_DIM, kernel_initializer=weight_initializer,
                      bias_initializer=bias_initializer)(hidden_layer)
generator = keras.Model(inputs=latent_sample, outputs = output, name="generator")
print('Generator:')
print(generator.summary())

Generator:
Model: "generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 4)]               0         
_________________________________________________________________
dense (Dense)                (None, 64)                320       
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_3 (Dense)              (None, 32)                2080      
Total params: 10,720
Trainable params: 10,720
Non-trainable params: 0
_________________________________________________________________
None


In [6]:
from scipy.special import logsumexp

In [7]:
def squared_l2_matrix_np(X, Y):
    squared_l2_X = np.sum(np.square(X), axis = 1, keepdims=True)
    squared_l2_Y = np.sum(np.square(Y), axis = 1, keepdims=True)
    XY = X.dot(Y.T)
    return squared_l2_X + tf.transpose(squared_l2_Y) - 2 * XY

def log_coupling_np(psi_X, psi_Y, cost_matrix, epsilon):
    C_tild = cost_matrix - np.expand_dims(psi_X, axis=1) - np.expand_dims(psi_Y, axis=0)
    return -C_tild/epsilon

def sinkhorn_step_np(psi_X, psi_Y, log_X_prob, log_Y_prob, cost_matrix, epsilon, return_diff = False):
    psi_X_upd = epsilon * (log_X_prob - \
                       logsumexp((np.expand_dims(psi_Y, axis=0) - cost_matrix) / epsilon, axis = 1))
    psi_Y_upd = epsilon * (log_Y_prob - \
                       logsumexp((np.expand_dims(psi_X_upd, axis=1) - cost_matrix)/epsilon,axis = 0))
    if return_diff:
        diff = np.linalg.norm(psi_X_upd - psi_X) + np.linalg.norm(psi_Y_upd - psi_Y)
    psi_X = psi_X_upd
    psi_Y = psi_Y_upd
    if return_diff:
        return psi_X, psi_Y, diff
    return psi_X, psi_Y

def sinkhorn_loss_np(X, Y, epsilon, num_steps):
    cost_matrix = squared_l2_matrix_np(X,Y)
    log_X_prob = - np.log(X.shape[0])
    log_Y_prob = - np.log(Y.shape[0])
    psi_Y = np.zeros(Y.shape[0])
    for l in range(num_steps-1):
        psi_X, psi_Y = sinkhorn_step_np(None, psi_Y, log_X_prob, log_Y_prob, cost_matrix, epsilon)
    psi_X, psi_Y, diff = sinkhorn_step_np(psi_X, psi_Y, log_X_prob, log_Y_prob, \
                                       cost_matrix, epsilon, return_diff = True)
    #return psi_X, psi_Y, diff
    log_pi = log_coupling_np(psi_X, psi_Y, cost_matrix, epsilon)
    pi = np.exp(log_pi)
    loss = np.sum(pi*(cost_matrix+epsilon*log_pi)) - epsilon*(log_X_prob + log_Y_prob)
    #X and Y are uniform, so entropy = -log(probability)
    return loss, diff

In [8]:
def squared_l2_matrix(X, Y):
    squared_l2_X = tf.reduce_sum(tf.square(X), axis = 1, keepdims=True)
    squared_l2_Y = tf.reduce_sum(tf.square(Y), axis = 1, keepdims=True)
    XY = tf.matmul(X,tf.transpose(Y))
    return squared_l2_X + tf.transpose(squared_l2_Y) - 2 * XY

def log_coupling(psi_X, psi_Y, cost_matrix, epsilon):
    C_tild = cost_matrix - tf.expand_dims(psi_X, axis=1) - tf.expand_dims(psi_Y, axis=0)
    return -C_tild/epsilon

def sinkhorn_step(psi_X, psi_Y, log_X_prob, log_Y_prob, cost_matrix, epsilon, return_diff = False):
    psi_X_upd = epsilon * (log_X_prob - \
                       tf.reduce_logsumexp((tf.expand_dims(psi_Y, axis=0) - cost_matrix) / epsilon, axis = 1))
    psi_Y_upd = epsilon * (log_Y_prob - \
                       tf.reduce_logsumexp((tf.expand_dims(psi_X_upd, axis=1) - cost_matrix)/epsilon,axis = 0))
    if return_diff:
        diff = tf.norm(psi_X_upd - psi_X) + tf.norm(psi_Y_upd - psi_Y)
    psi_X = psi_X_upd
    psi_Y = psi_Y_upd
    if return_diff:
        return psi_X, psi_Y, diff
    return psi_X, psi_Y

def sinkhorn_loss(X, Y, epsilon, num_steps, return_diff = False, return_diff_only = False):
    cost_matrix = squared_l2_matrix(X,Y)
    log_X_prob = - tf.math.log(tf.cast(tf.shape(X)[0], tf.float32))
    log_Y_prob = - tf.math.log(tf.cast(tf.shape(Y)[0], tf.float32))
    psi_Y = tf.zeros([tf.shape(Y)[0]])
    for l in range(num_steps-1):
        psi_X, psi_Y = sinkhorn_step(None, psi_Y, log_X_prob, log_Y_prob, cost_matrix, epsilon)
    psi_X, psi_Y, diff = sinkhorn_step(psi_X, psi_Y, log_X_prob, log_Y_prob, \
                                       cost_matrix, epsilon, return_diff = True)
    #return psi_X, psi_Y, diff
    log_pi = log_coupling(psi_X, psi_Y, cost_matrix, epsilon)
    pi = tf.exp(log_pi)
    loss = tf.reduce_sum(pi*(cost_matrix+epsilon*log_pi)) - epsilon*(log_X_prob + log_Y_prob)
    #X and Y are uniform, so entropy = -log(probability)
    if return_diff_only:
        return diff
    if return_diff:
        return loss, diff
    return loss

In [9]:
epsilon = LAMBDA
if MODE == 'divergence':
    model_loss = lambda x, y: sinkhorn_loss(x, y, epsilon, CRITIC_ITERS) - 0.5*\
        (sinkhorn_loss(x, x, epsilon, CRITIC_ITERS)-sinkhorn_loss(y, y, epsilon, CRITIC_ITERS))
else:
    model_loss = lambda x, y: sinkhorn_loss(x, y, epsilon, CRITIC_ITERS)

if MODE == 'divergence':
    args = [epsilon, CRITIC_ITERS, True, True]
    sinkhorn_diff = lambda x, y: (sinkhorn_loss(x, y, *args)\
        +sinkhorn_loss(x, x, *args)+sinkhorn_loss(y, y, *args))/3
else:
    sinkhorn_diff = lambda x, y: sinkhorn_loss(x, y, epsilon, CRITIC_ITERS)

metrics = lambda x, y: tf.norm(tfp.stats.covariance(x) - tfp.stats.covariance(y))
generator.compile(optimizer="Adam", loss=model_loss, metrics=metrics)

In [10]:
def inf_train_gen():
    np.random.seed(1)
    full_dataset = np.random.randn(SAMPLE_SIZE,DATA_DIM) / np.sqrt(COVARIANCE_SCALE) 
    i = 0
    offset = 0
    while True:
        dataset = full_dataset[i*BATCH_SIZE+offset:(i+1)*BATCH_SIZE+offset,:]
        if (i+1)*BATCH_SIZE+offset > SAMPLE_SIZE: 
            offset = (i+1)*BATCH_SIZE+offset - SAMPLE_SIZE
            np.random.shuffle(full_dataset)
            dataset = np.concatenate([dataset,full_dataset[:offset,:]], axis = 0)
            i = -1 
        i+=1
        yield dataset
data_gen = inf_train_gen()

In [12]:
y_batch = next(data_gen)
x_batch = np.random.normal(size=(y_batch.shape[0], LATENT_DIM))
res_train = {'loss':{}, 'cov_diff' : {}}
res_test = {'loss':{}, 'cov_diff' : {}, 'sample':{}, 'sink_eps':{}}
for i in range(ITERS):
    out = generator.train_on_batch(x = x_batch,y = y_batch)
    res_train['loss'][i] = out[0]
    res_train['cov_diff'][i] = out[1]
    y_batch = next(data_gen)
    x_batch = np.random.normal(size=(y_batch.shape[0], LATENT_DIM))
    #validate on the next data
    y_sample = generator.predict(x_batch, batch_size=BATCH_SIZE)
    res_test['loss'][i], res_test['sink_eps'][i] = sinkhorn_loss_np(y_sample, y_batch, epsilon, CRITIC_ITERS)
    res_test['sample'][i] = y_sample
    res_test['cov_diff'][i] = np.linalg.norm(np.cov(y_sample[:256,:]) - np.cov(y_batch[:256,:]))
    print('Iteration ', i)
    print('Training: loss {}, covariance difference {}'.format(res_train['loss'][i], res_train['cov_diff'][i]))
    print('Validation: loss {}, covariance difference {}, sinkhorn epsilon {}'.format(
        res_test['loss'][i], res_test['cov_diff'][i], res_test['sink_eps'][i]))
    if i % 10 == 0:
        with open('logs.pkl', 'wb') as f:
            pkl.dump([res_test, res_train], f)

Iteration  0
Training: loss 4.678339958190918, covariance difference 1.0191571712493896
Validation: loss 5.60212407967657, covariance difference 8.62967076942069, sinkhorn epsilon 0.00026994421971048867
Iteration  1
Training: loss 4.577845573425293, covariance difference 1.0029255151748657
Validation: loss 5.58790516611306, covariance difference 8.460720479334046, sinkhorn epsilon 3.248205632779033e-11
Iteration  2
Training: loss 4.562339782714844, covariance difference 0.9992423057556152
Validation: loss 5.581281204060176, covariance difference 8.395882684734616, sinkhorn epsilon 8.801552625700866e-08
Iteration  3
Training: loss 4.552487373352051, covariance difference 0.9983996748924255
Validation: loss 5.6549487054529095, covariance difference 8.494231741067493, sinkhorn epsilon 3.8858213705074945e-10
Iteration  4
Training: loss 4.627856254577637, covariance difference 1.0117124319076538
Validation: loss 5.718280632689711, covariance difference 8.689667058771258, sinkhorn epsilon 2.

Iteration  41
Training: loss 4.669532775878906, covariance difference 1.0246940851211548
Validation: loss 5.604947636871195, covariance difference 8.266364896379814, sinkhorn epsilon 3.4610760049220496e-14
Iteration  42
Training: loss 4.5683135986328125, covariance difference 1.0040119886398315
Validation: loss 5.616511702321495, covariance difference 8.543063073719352, sinkhorn epsilon 1.2189317731430368e-14
Iteration  43
Training: loss 4.579812049865723, covariance difference 1.0052196979522705
Validation: loss 5.656889041374058, covariance difference 8.915047515354901, sinkhorn epsilon 2.346716343773844e-13
Iteration  44
Training: loss 4.620477199554443, covariance difference 1.0139461755752563
Validation: loss 5.708619079343099, covariance difference 8.713153250121515, sinkhorn epsilon 2.0353743076872016e-14
Iteration  45
Training: loss 4.671381950378418, covariance difference 1.0230929851531982
Validation: loss 5.748954251049458, covariance difference 8.776916813466269, sinkhorn e

Iteration  81
Training: loss 4.637916564941406, covariance difference 1.0177866220474243
Validation: loss 5.695385863917938, covariance difference 8.727246345844677, sinkhorn epsilon 3.417184930139891e-14
Iteration  82
Training: loss 4.656761646270752, covariance difference 1.0209609270095825
Validation: loss 5.687597215367942, covariance difference 8.636706367598565, sinkhorn epsilon 2.590385200531446e-15
Iteration  83
Training: loss 4.6490960121154785, covariance difference 1.020076870918274
Validation: loss 5.686344790644112, covariance difference 8.613278395725294, sinkhorn epsilon 1.334607041491032e-15
Iteration  84
Training: loss 4.647243976593018, covariance difference 1.0221680402755737
Validation: loss 5.638856645331748, covariance difference 8.513427927873858, sinkhorn epsilon 3.088795318704387e-14
Iteration  85
Training: loss 4.600470542907715, covariance difference 1.012421727180481
Validation: loss 5.570053248105544, covariance difference 8.530423130247806, sinkhorn epsilo

Iteration  122
Training: loss 4.49701452255249, covariance difference 0.9938242435455322
Validation: loss 5.681553071730638, covariance difference 8.577952459460251, sinkhorn epsilon 0.0
Iteration  123
Training: loss 4.642490386962891, covariance difference 1.0172357559204102
Validation: loss 5.715524533258426, covariance difference 8.90490937148199, sinkhorn epsilon 0.0
Iteration  124
Training: loss 4.676111221313477, covariance difference 1.024778127670288
Validation: loss 5.661212502941945, covariance difference 8.758997182520309, sinkhorn epsilon 4.044954707664469e-14
Iteration  125
Training: loss 4.621967792510986, covariance difference 1.0155813694000244
Validation: loss 5.631757943524001, covariance difference 8.612479413222795, sinkhorn epsilon 3.3864672621763756e-14
Iteration  126
Training: loss 4.592863082885742, covariance difference 1.008902907371521
Validation: loss 5.79035310409523, covariance difference 8.729532365236244, sinkhorn epsilon 2.7755575615628914e-16
Iteration

Iteration  163
Training: loss 4.615317344665527, covariance difference 1.0150104761123657
Validation: loss 5.660746729425448, covariance difference 8.77425165742974, sinkhorn epsilon 3.571861599547592e-14
Iteration  164
Training: loss 4.621325492858887, covariance difference 1.0168911218643188
Validation: loss 5.729930953070834, covariance difference 8.980283172904025, sinkhorn epsilon 0.0
Iteration  165
Training: loss 4.690426826477051, covariance difference 1.0270447731018066
Validation: loss 5.682201181822675, covariance difference 8.577154178310003, sinkhorn epsilon 1.4790793018918976e-15
Iteration  166
Training: loss 4.642974376678467, covariance difference 1.0194635391235352
Validation: loss 5.632280210565149, covariance difference 8.694287109425526, sinkhorn epsilon 0.0
Iteration  167
Training: loss 4.592820167541504, covariance difference 1.0085008144378662
Validation: loss 5.648914007794456, covariance difference 8.663784009390676, sinkhorn epsilon 3.808532904460161e-15
Iterat

...

Iteration  2290
Training: loss 4.598793983459473, covariance difference 1.012137770652771
Validation: loss 5.6887424237997495, covariance difference 8.60341077858073, sinkhorn epsilon 0.0
Iteration  2291
Training: loss 4.648996353149414, covariance difference 1.0212842226028442
Validation: loss 5.617506468180696, covariance difference 8.541341932672436, sinkhorn epsilon 0.0
Iteration  2292
Training: loss 4.577783107757568, covariance difference 1.0101581811904907
Validation: loss 5.640428222297895, covariance difference 8.659582509974264, sinkhorn epsilon 0.0
Iteration  2293
Training: loss 4.6006927490234375, covariance difference 1.0127990245819092
Validation: loss 5.758347971724494, covariance difference 8.748247219223138, sinkhorn epsilon 0.0
Iteration  2294
Training: loss 4.718611240386963, covariance difference 1.0342893600463867
Validation: loss 5.665646426464321, covariance difference 8.398031368754612, sinkhorn epsilon 0.0
Iteration  2295
Training: loss 4.625913619995117, covar

Iteration  2334
Training: loss 4.662641525268555, covariance difference 1.0231022834777832
Validation: loss 5.701305305814554, covariance difference 8.759137245773104, sinkhorn epsilon 0.0
Iteration  2335
Training: loss 4.661563873291016, covariance difference 1.024367094039917
Validation: loss 5.574679270090948, covariance difference 8.420264494508244, sinkhorn epsilon 0.0
Iteration  2336
Training: loss 4.53494930267334, covariance difference 1.0018068552017212
Validation: loss 5.624599661210737, covariance difference 8.48198704020123, sinkhorn epsilon 3.6559965400591467e-14
Iteration  2337
Training: loss 4.584867477416992, covariance difference 1.0094765424728394
Validation: loss 5.5782196596070905, covariance difference 8.494167829476572, sinkhorn epsilon 0.0
Iteration  2338
Training: loss 4.5384840965271, covariance difference 1.0027610063552856
Validation: loss 5.671544116790415, covariance difference 8.66042480019916, sinkhorn epsilon 0.0
Iteration  2339
Training: loss 4.63182163

Iteration  2378
Training: loss 4.635239601135254, covariance difference 1.0188673734664917
Validation: loss 5.699393717895793, covariance difference 8.740717218226541, sinkhorn epsilon 0.0
Iteration  2379
Training: loss 4.659665107727051, covariance difference 1.0226683616638184
Validation: loss 5.591391546048985, covariance difference 8.321619320971442, sinkhorn epsilon 0.0
Iteration  2380
Training: loss 4.5516510009765625, covariance difference 1.0040675401687622
Validation: loss 5.684167169426539, covariance difference 8.441722748100048, sinkhorn epsilon 0.0
Iteration  2381
Training: loss 4.644435405731201, covariance difference 1.0193636417388916
Validation: loss 5.687574746627506, covariance difference 8.890537106614861, sinkhorn epsilon 0.0
Iteration  2382
Training: loss 4.647839546203613, covariance difference 1.0212602615356445
Validation: loss 5.688210840614331, covariance difference 8.843014788167292, sinkhorn epsilon 0.0
Iteration  2383
Training: loss 4.648478984832764, cova

Iteration  2422
Training: loss 4.647294521331787, covariance difference 1.0192958116531372
Validation: loss 5.684888491415287, covariance difference 8.57615541038302, sinkhorn epsilon 0.0
Iteration  2423
Training: loss 4.6451568603515625, covariance difference 1.0212184190750122
Validation: loss 5.660691554018415, covariance difference 8.707924446002162, sinkhorn epsilon 0.0
Iteration  2424
Training: loss 4.620955467224121, covariance difference 1.0167316198349
Validation: loss 5.640913172004689, covariance difference 8.324815180079534, sinkhorn epsilon 0.0
Iteration  2425
Training: loss 4.6011810302734375, covariance difference 1.0130890607833862
Validation: loss 5.625791612425095, covariance difference 8.629652004427225, sinkhorn epsilon 0.0
Iteration  2426
Training: loss 4.5860595703125, covariance difference 1.0100317001342773
Validation: loss 5.599217198336726, covariance difference 8.700639888754527, sinkhorn epsilon 0.0
Iteration  2427
Training: loss 4.5594868659973145, covarian

Iteration  2466
Training: loss 4.696832656860352, covariance difference 1.027296543121338
Validation: loss 5.647126422009386, covariance difference 8.414685723337527, sinkhorn epsilon 0.0
Iteration  2467
Training: loss 4.607394218444824, covariance difference 1.0132899284362793
Validation: loss 5.685137460283099, covariance difference 8.885702387228484, sinkhorn epsilon 0.0
Iteration  2468
Training: loss 4.645401477813721, covariance difference 1.0212925672531128
Validation: loss 5.667649714894838, covariance difference 8.714643387043603, sinkhorn epsilon 0.0
Iteration  2469
Training: loss 4.6279144287109375, covariance difference 1.0160603523254395
Validation: loss 5.669322039824641, covariance difference 8.745205860435366, sinkhorn epsilon 0.0
Iteration  2470
Training: loss 4.629574775695801, covariance difference 1.0190370082855225
Validation: loss 5.679899005635677, covariance difference 8.747844418415694, sinkhorn epsilon 0.0
Iteration  2471
Training: loss 4.640162467956543, covar

In [ ]:
x, y = zip(*list(res_test['cov_diff'].items()))

start = 100
plt.plot(x[start:], y[start:])